In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

import torch.nn.functional as F

`torchvision.transforms` 是 pytorch 中的图像预处理包。一般用Compose把多个步骤整合到一起

Normalize：Normalized an tensor image with mean and standard deviation

ToTensor：convert a PIL image to tensor $(H*W*C)$ in range $[0,255]$ to a torch.Tensor $(C*H*W)$ in the range $[0.0,1.0]$


In [ ]:
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), # 将图片格式转换为 tensor
                transforms.Normalize((0.1307,), (0.3081,))]) # 归一化

train_dataset = datasets.MNIST(root='../dataset/mnist/',
                  train=True,
                  transform=transform,
                  download=True)
test_dataset = datasets.MNIST(root='../dataset/mnist/',
                  train=False,
                  transform=transform,
                  download=True)
train_loader = DataLoader(dataset=train_dataset, 
              batch_size=batch_size,
              shuffle=True,
              )
test_loader = DataLoader(dataset=test_dataset,
              batch_size=batch_size,
              shuffle=False,
              )

In [3]:
class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.linear1 = torch.nn.Linear(784, 512)
    self.linear2 = torch.nn.Linear(512, 256)
    self.linear3 = torch.nn.Linear(256, 128)
    self.linear4 = torch.nn.Linear(128, 64)
    self.linear5 = torch.nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(-1, 784) # 输入的维度是(N,1,28,28)，但是在全连接网络里，需要变为一阶的向量（N，784），使用x=x.view(-1,784),这里的-1是指根据784自动算出的维度
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = F.relu(self.linear3(x))
    x = F.relu(self.linear4(x))

    return self.linear5(x) # 最后一层不做激活，在使用交叉熵CELoss作为损失函数时，会自动调用softmax函数将最后一层激活

model = Model()

In [4]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5) # momentum动量，可以理解为给物体一个惯性，容易越过局部极小点，一般设置为0.5

In [5]:
def train(epoch):
  loss_sum = 0
  for i, (inputs, labels) in enumerate(train_loader, 0):
    y_pred = model(inputs)
    loss = criterion(y_pred, labels)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    loss_sum += loss.item()
    if i % 300 == 299: # 每300个batch输出损失
      print("[%d, %5d] loss:%.3f" % (epoch+1, i+1, loss_sum/300))
      loss = 0

`torch.max()` 第一个参数返回指定行的最大值，第二个参数返回指定行最大值的下标

In [6]:
def test():
  correct = 0
  total = 0
  # 测试集不计算梯度
  with torch.no_grad():
    for (inputs, labels) in test_loader:
      y_pred = model(inputs)
      _, predicted = torch.max(y_pred.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
      total += batch_size 
      correct += (predicted == labels).sum().item() # 
  print('accuracy on test set: %d %% ' % (100*correct/total))

In [7]:
if __name__ == '__main__':
  for epoch in range(10):
    train(epoch)
    test()

[1,   300] loss:2.259
[1,   600] loss:3.348
[1,   900] loss:3.772
accuracy on test set: 88 % 
[2,   300] loss:0.312
[2,   600] loss:0.592
[2,   900] loss:0.830
accuracy on test set: 93 % 
[3,   300] loss:0.194
[3,   600] loss:0.364
[3,   900] loss:0.521
accuracy on test set: 95 % 
[4,   300] loss:0.136
[4,   600] loss:0.258
[4,   900] loss:0.375
accuracy on test set: 95 % 
[5,   300] loss:0.101
[5,   600] loss:0.192
[5,   900] loss:0.285
accuracy on test set: 96 % 
[6,   300] loss:0.074
[6,   600] loss:0.152
[6,   900] loss:0.229
accuracy on test set: 96 % 
[7,   300] loss:0.060
[7,   600] loss:0.120
[7,   900] loss:0.185
accuracy on test set: 96 % 
[8,   300] loss:0.045
[8,   600] loss:0.097
[8,   900] loss:0.150
accuracy on test set: 97 % 
[9,   300] loss:0.041
[9,   600] loss:0.084
[9,   900] loss:0.126
accuracy on test set: 97 % 
[10,   300] loss:0.031
[10,   600] loss:0.065
[10,   900] loss:0.097
accuracy on test set: 97 % 
